### 데이터 로딩 및 라이브러리 import

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import Image  # 주피터 노트북에 이미지 삽입
import seaborn as sns
import matplotlib.ticker as ticker # plt 축의 tik(눈금, 틱) 

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('Data/diabetes.csv')
df.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


* Pregnancies : 임신횟수  
* Glucose : 경구 포도당 내성검사 2시간 혈장 포도당 농도 (140미만 정상, 140-199 내당능장애, 200이상 당뇨병)  
* BloodPressure : 이완기 혈압 (mmHg)
* SkinThickness : 팔 삼두근 뒤쪽의 피하지방 측정값 (mm)
* Insulin : 2시간 혈청 인슐린(mu U/mL)
* BMI : 체질량지수(체중(kg) / 키(cm)^2)
* DiabetesPedigreeFunction : 당뇨 직계 가족력
* Age : 나이
* Outcome : 5년이내 당뇨병 발병여부 (0 또는 1)    

In [3]:
### 데이터 전처리 
* df.info() : length, not null(결측치), data type
* df.describe() : 4분위, 통계(statics)
* plot : 시각적 분포(histogram,swarmplot, boxplot(이상치))
* zero 0값 찾기
#### ==> pandas_profiling

SyntaxError: invalid syntax (<ipython-input-3-abb7779ad9ea>, line 2)

## 정규분포 검사

In [4]:
#scipy.stats에서 샤피로 가져 오기
from scipy.stats import shapiro , normaltest , anderson , kstest
stat, p = shapiro (df[ 'BloodPressure']) 
print (f'통계:{stat:.3f} p값:{p:.3f}')

통계:0.819 p값:0.000


#### shapiro

In [5]:
normal = []
notnormal = []
num_var= ['Pregnancies','Glucose', 'BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction', 'Age', 'Outcome' ]
for var in num_var :
#     stat, p  = shapiro(df[var].dropna().values)  # zero column 삭제
#     stat, p  = normaltest(df[var].dropna().values)  # zero column 삭제
    
    stat, p  = shapiro(df[var].fillna(value=df[var].median()).values) # 중앙값 대체
#     stat, p  = normaltest(df[var].fillna(value=df[var].median()).values)# 중앙값 대체
    
    alpha = 0.05
    if p > alpha :
        normal.append(var)
    else :
        notnormal.append(var)
print(f'normal:  {normal}')
print(f'notnormal:  {notnormal}')

normal:  []
notnormal:  ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']


#### kstest

In [6]:
normal = []
notnormal = []
num_var= ['Pregnancies','Glucose', 'BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction', 'Age', 'Outcome' ]
for var in num_var :
    stat,p  = kstest(df[var].dropna().values, "norm")
#     stat,p  = kstest(df[var].fillna(value=df[var].median()).values, "norm")
    alpha = 0.05
    if p > alpha :
        normal.append(var)
    else :
        notnormal.append(var)
print(f'normal:  {normal}')
print(f'notnormal:  {notnormal}')

normal:  []
notnormal:  ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']


## VIF를 통한 다중공선선 확인

In [ ]:
* VIF는 variance inflation factor의 줄임말로, 다중공선성을 확인할 때 쓰는 지표 중 하나다.  
* variance inflation factor는 말그대로 "분산팽창요인"이다. 보통은 VIF가 10보다 크면 다중공선성이 있다고 판단한다.  
* 하지만, 다른 과정을 함께 거쳐주는 것이 다중공선성 문제 확인의 신뢰성을 높인다.  

In [12]:
import statsmodels.api as sm
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor 

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
vif["features"] = df.columns
vif

,VIF Factor,features
0,3.364416,Pregnancies
1,18.030209,Glucose
2,15.313159,BloodPressure
3,4.008709,SkinThickness
4,2.063940,Insulin
5,18.515112,BMI
6,3.213450,DiabetesPedigreeFunction
7,13.500531,Age
8,1.948398,Outcome


In [ ]:
#### 아래는 boston 사례임

In [ ]:
* RM, B, TAX, AGE, NOX, INDUS 변수들의 VIF가 10보다 크다. 
* vif가 높은 변수가 하나씩 줄어들면 다른 변수들의 vif에도 영향을 미친다.  
* 그래서 변수들을 한 번에 다 제거하기보다는 하나씩 제거하면서 확인해 나아가는 것이 바람직하다. 

In [ ]:
# nox 변수 제거 후 vif 확인

vif = pd.DataFrame()
x_data3 = x_data2.drop("NOX",axis=1)
vif["VIF Factor"] = [variance_inflation_factor(x_data3.values, i) for i in range(x_data3.shape[1])]
vif["features"] = x_data3.columns
vif

In [ ]:
* 전반적으로 약간씩 VIF가 줄어들었다. 하지만 RM은 여전히 매우 높기에 확실하게 제거할 필요가 있다.  

In [ ]:
# RM 변수 제거후 VIF확인

vif = pd.DataFrame()
x_data4 = x_data3.drop("RM",axis = 1)
vif["VIF Factor"] = [variance_inflation_factor(x_data4.values, i) for i in range(x_data4.shape[1])]
vif["features"] = x_data4.columns
vif

In [ ]:
# nox 변수를 제거한 x_data3 상수항 추가 후 회귀모델 적합시키기
# nox, rm 변수를 제거한 x_data4 상수항 추가 후 회귀모델 적합시키기

x_data3_ = sm.add_constant(x_data3, has_constant = "add")
x_data4_ = sm.add_constant(x_data4, has_constant = "add")

model_vif = sm.OLS(target, x_data3_)
fitted_model_vif = model_vif.fit()

model_vif2 = sm.OLS(target,x_data4_)
fitted_model_vif = model_vif2.fit()

# 두 vif를 통한 변수제거 회귀모델의 결과를 비교

fitted_model_vif.summary()

In [ ]:
* 맨 처음 모든 변수를 다 더했을 때 OLS는 condition number가 1.04e+04 였는데 
* nox변수를 지우고 나서 8.44e+03으로 떨어진 것을 확인 할 수 있다. 다중공선성이 조금은 완화된 것을 확인할 수 있다  

In [ ]:
* 위는 NOX와 RM 모두 제거 후 OLS검정 결과인데, 여기서 condition number는 3.85e+03으로 훨씬 더 낮아졌다.  
* 물론 그렇다고 다중공선성이 완전하게 사라진 것은 아니다. 그래도 앞의 모델보다는 더 나은 결과를 보여준다.  

### 데이터 분할 및 학습

In [ ]:
# 학습 검증데이터 분할
from sklearn.model_selection import train_test_split

X = x_data2_
y = target

train_x, test_x, train_y, test_y = train_test_split(X,y, train_size = 0.7, test_size = 0.3, random_state = 1)
# 학습데이터와 검증데이터를 7:3으로 분리한다.
# random_state고정을 통해 그때마다 똑같은 값을 분류하도록 한다.

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

In [ ]:
# train_x에 상수항 추가 후 최귀모델 적합

fit_train1 = sm.OLS(train_y,train_x)
fit_train1 = fit_train1.fit()

# 검증데이터에 대한 예측값과 true값 비교

plt.plot(np.array(fit_train1.predict(test_x)),label = "pred")
plt.plot(np.array(test_y),label = "true")
plt.legend()
plt.show()

In [ ]:
# x_data3와 x_data4 학습 검증데이터 분할

X = x_data3_
y = target
train_x2,test_x2,train_y2,test_y2 = train_test_split(X,y,train_size=0.7, test_size=0.3, random_state=1)

X = x_data4_
y = target
train_x3,test_x3,train_y3,test_y3 = train_test_split(X,y,train_size=0.7, test_size=0.3, random_state=1)

# x_data3/x_data4의 회귀모델 적합 (fit_train2,fit_train3)

fit_train2 = sm.OLS(train_y2,train_x2)
fit_train2 = fit_train2.fit()

fit_train3 = sm.OLS(train_y3,train_x3)
fit_train3 = fit_train3.fit()

# vif를 통해 NOX를 지운 데이터 x_data3 , NOX,RM을 지운 데이터 x_data4 full모델 실제값 비교

plt.plot(np.array(fit_train1.predict(test_x)),label = "pred_full")
plt.plot(np.array(fit_train2.predict(test_x2)),label = "pred_vif")
plt.plot(np.array(fit_train3.predict(test_x3)),label = "pred_vif2")
plt.plot(np.array(test_y2), label = "true")
plt.legend()
plt.show()

In [ ]:
### MSE

In [ ]:
from sklearn.metrics import mean_squared_error

#변수 제거가 이루어지지 않은 full모델
mse1 = mean_squared_error(y_true = test_y["Target"], y_pred = fit_train1.predict(test_x))

#변수 NOX만 제거한 모델
mse2 = mean_squared_error(y_true = test_y["Target"], y_pred = fit_train2.predict(test_x2))

#변수 NOX와 RM 두 개를 제거한 모델
mse3 = mean_squared_error(y_true = test_y["Target"], y_pred = fit_train3.predict(test_x3))

print(mse1)
print(mse2)
print(mse3)
